In [1]:
import numpy as np
from feature_treatment import load_images_from_folder, feature_extraction_set, feature_matching_set, initialize_reconstruction
from utils import connect_images, fill_pts3d, plot_model
from read_write_model import rotmat2qvec, qvec2rotmat, Camera
from camera_treatment import calculate_projection_matrix, reprojection_error
from triangulation import following_img_reconstruction, alignment_of_kpts, triangulate_and_reproject
from bundle_adjustment import bundleAd

In [2]:
cameras = {}
punts3D = {}

In [3]:
images, img_db = load_images_from_folder('dinos')

In [4]:
 # Feature extraction
kp, des = feature_extraction_set(images, img_db)

# Feature matching
matches = feature_matching_set(kp, des)

In [5]:
adj_matrix = connect_images(matches)

In [6]:
# Inicializar camara
height, width = images.shape[1:3]
K = np.array([  # for dino
    [2360, 0, width / 2],
    [0, 2360, height / 2],
    [0, 0, 1]])

cameras[1] = Camera(
                id=1,
                model="OPENCV",
                width=width,
                height=height,
                params=[2360, 2360, width / 2, height / 2, 0,0,0,0],
            )
punts3D = {}

In [7]:
initial_pair, RT1, RT2, pts_cloud = initialize_reconstruction(adj_matrix, matches, kp, K)
img_db[initial_pair[0]].qvec = rotmat2qvec(RT1[:3, :3])
img_db[initial_pair[0]].tvec = RT1[:3, -1]
img_db[initial_pair[1]].qvec = rotmat2qvec(RT2[:3, :3])
img_db[initial_pair[1]].tvec = RT2[:3, -1]
fill_pts3d(img_db, punts3D, pts_cloud, kp[initial_pair[0]], matches, images, 0, 1)
plot_model(punts3D)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
processed_imgs = [initial_pair[0], initial_pair[1]]
unprocessed_imgs = [i for i in range(len(images)) if not i in processed_imgs]

In [ ]:
BA_chkpts = [3,4,5,6] + [int(6*(1.34**i)) for i in range(25)]
while len(unprocessed_imgs) > 0:
    processed_idx, unprocessed_idx, prepend = following_img_reconstruction(images.shape[0], initial_pair, processed_imgs, unprocessed_imgs)
    pts3d_idx = img_db[unprocessed_idx + 1].point3D_idxs != -1
    if np.sum(pts3d_idx) < 12:
        continue
    
    pts3d = img_db[unprocessed_idx + 1].point3D_idxs[pts3d_idx]
    pts2d = img_db[unprocessed_idx + 1].xys[pts3d_idx]
    
    R1 = qvec2rotmat(img_db[unprocessed_idx + 1].qvec)
    T1 = img_db[unprocessed_idx + 1].tvec
    RT1 = np.hstack((R1, T1))
    RT2 = calculate_projection_matrix(K, pts3d, pts2d)
    
    img_db[unprocessed_idx + 1].qvec = rotmat2qvec(RT2[:3, :3])
    img_db[unprocessed_idx + 1].tvec = RT2[:3, -1]

    if prepend: processed_imgs.insert(0, unprocessed_idx)
    else: processed_imgs.append(unprocessed_idx)
    unprocessed_imgs.remove(unprocessed_idx)
    _, perc_inliers = reprojection_error(K, RT1, pts3d, pts2d)
    
    if processed_idx < unprocessed_idx:
        kpts1, kpts2, kpts1_idx, kpts2_idx = alignment_of_kpts(kp, matches, processed_idx, unprocessed_idx)
        if len(kpts1) > 0:
            pts_cloud, avg_tri_err_l, avg_tri_err_r = triangulate_and_reproject(K, RT1, K, RT2, kpts1, kpts2)
    else:
        kpts1, kpts2, kpts1_idx, kpts2_idx = alignment_of_kpts(kp, matches, unprocessed_idx, processed_idx)
        if len(kpts1) > 0:
            pts_cloud, avg_tri_err_l, avg_tri_err_r = triangulate_and_reproject(K, RT2, K, RT1, kpts1, kpts2)
        
    if 0.8 < perc_inliers < 0.95 or 5 < avg_tri_err_l < 10 or 5 < avg_tri_err_r < 10: 
        bundleAd(punts3D, img_db, processed_imgs, K, ftol=1e0)
        
    if len(processed_imgs) in BA_chkpts or len(unprocessed_imgs) == 0 or perc_inliers <= 0.8 or avg_tri_err_l >= 10 or avg_tri_err_r >= 10:
        bundleAd(punts3D, img_db, processed_imgs, K, ftol=1e-1)

In [ ]:
plot_model(punts3D)

In [ ]:
# for i in range(2, len(img_db)):
#     # Get projection matrix, rotation and translation
#     pts2d = np.array([pt for pt, idx in zip(img_db[i+1].xys, img_db[i+1].point3D_idxs) if idx != -1])
#     pts3d = np.array([punts3D[idx].xyz for idx in img_db[i+1].point3D_idxs if idx != -1])
#     P = calculate_projection_matrix(K, pts3d, pts2d)
#     RT = np.linalg.inv(K) @ P
#     img_db[i+1].qvec = rotmat2qvec(RT[:, :3])
#     img_db[i+1].tvec = RT[:, -1]
    
#     # Triangulate all matches with previous images and
#     for j in range(i):
#         if len(matches[(j,i)]) == 0:
#             continue
        
#         pts2d1_idxs = np.array([m.queryIdx for m in matches[(j,i)] if img_db[i+1].point3D_idxs[m.trainIdx] == -1]).astype(int)
#         pts2d2_idxs = np.array([m.trainIdx for m in matches[(j,i)] if img_db[i+1].point3D_idxs[m.trainIdx] == -1]).astype(int)

#         RTj = np.hstack((qvec2rotmat(img_db[j+1].qvec), img_db[j+1].tvec[:, np.newaxis]))
#         RTi = RT
        
#         pts_cloud = linear_triangulation(K, RTj, K, RT, img_db[j+1].xys[pts2d1_idxs, :].T, img_db[i+1].xys[pts2d2_idxs, :].T)

#         fill_pts3d(img_db, punts3D, pts_cloud, kp[j], matches, images, j, i)
